In [10]:
'''
    Load RESNET model and split it
        1. layer by layer
        2. [TODO] vertically 
'''

'\n    Load RESNET model and split it\n        1. layer by layer\n        2. [TODO] vertically \n'

In [11]:
from source.core.engine import MoP
from source.core import run_partition
from os import environ
from source.utils.dataset import *
from source.utils.misc import *
from source.utils.split_network import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from source.models import resnet
from source.core.split_manager import SplitManager

import torch.nn.functional as F

import numpy as np

from source.utils import io
from source.utils import testers
from source.core import engine

import json
import itertools

from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchsummary import summary

import time

In [12]:
# setup config
dataset='cifar10'
environ["config"] = f"config/{dataset}.yaml"

configs = run_partition.main()

configs["device"] = "cpu"
configs['load_model'] = "cifar10-resnet18-kernel-npv2-pr0.75-lcm0.001.pt"
configs["num_partition"] = '4' #'resnet18-v2.yaml'

device :  
model :  resnet18
data_code :  cifar10
num_classes :  10
model_file :  test.pt
epochs :  0
batch_size :  128
optimizer :  sgd
lr_scheduler :  default
learning_rate :  0.01
seed :  1234
sparsity_type :  kernel
prune_ratio :  1
admm :  True
admm_epochs :  300
rho :  0.0001
multi_rho :  True
retrain_bs :  128
retrain_lr :  0.005
retrain_ep :  50
retrain_opt :  default
xentropy_weight :  1.0
warmup :  False
warmup_lr :  0.001
warmup_epochs :  10
mix_up :  True
alpha :  0.3
smooth :  False
smooth_eps :  0
save_last_model_only :  False
num_partition :  1
layer_type :  regular
bn_type :  masked
par_first_layer :  False
comm_outsize :  False
lambda_comm :  0
lambda_comp :  0
distill_model :  
distill_loss :  kl
distill_temp :  30
distill_alpha :  1


In [13]:
# load data and load or train model
model = get_model_from_code(configs).to(configs['device']) # grabs model architecture from ./source/models/escnet.py
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [14]:
# load weights into full model
state_dict = torch.load(io.get_model_path("{}".format(configs["load_model"])), map_location=configs['device'])
model = io.load_state_dict(model, 
                    state_dict['model_state_dict'] if 'model_state_dict' in state_dict 
                    else state_dict['state_dict'] if 'state_dict' in state_dict else state_dict,)


In [15]:
'''
    add partitions and communications to configs
'''

# gets random test input (with correct size)
input_var = engine.get_input_from_code(configs)
#print(input_var)

# Config partitions and prune_ratio
configs = engine.partition_generator(configs, model)
            
# Compute collected_data_queues size of each layer
configs['partition'] = engine.featuremap_summary(model, configs['partition'], input_var)

# split model general parameters

# make copies of model per machine
num_machines = max(configs['partition']['bn_partition']) # TODO: double check this makes sense
model_machines = [model]*num_machines

layer_names_fx =  get_graph_node_names(model)[1]
total_layers_fx = len(layer_names_fx)

split_module_names = list(configs['partition'].keys())

print(layer_names_fx)


Inference time per data is 32.539129ms.
conv1.weight 1024
layer1.0.conv1.weight 1024
layer1.0.conv2.weight 1024
layer1.1.conv1.weight 1024
layer1.1.conv2.weight 1024
layer2.0.conv1.weight 256
layer2.0.conv2.weight 256
layer2.0.shortcut.0.weight 256
layer2.1.conv1.weight 256
layer2.1.conv2.weight 256
layer3.0.conv1.weight 64
layer3.0.conv2.weight 64
layer3.0.shortcut.0.weight 64
layer3.1.conv1.weight 64
layer3.1.conv2.weight 64
layer4.0.conv1.weight 16
layer4.0.conv2.weight 16
layer4.0.shortcut.0.weight 16
layer4.1.conv1.weight 16
layer4.1.conv2.weight 16
['x', 'conv1', 'bn1', 'relu', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.add', 'layer1.0.relu_1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.add', 'layer1.1.relu_1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.shortcut.0', 'layer2.0.shortcut.1', 'layer2.0.add', 'layer2.0.relu_1', 'la

In [16]:
''' 
    IO helpers
'''

def prep_data(tensor, node, layer):
    '''
        Prepare input tensors to model 
    '''
    return {
        'node' : node,
        'layer' : layer,
        'tensor' : tensor, 
        'Cin' : list(get_nonzero_channels(tensor)),
        'is_empty' : False
    }

def compare_outputs_wrapper(horz_output, comm_layer, split_model_output, limit = 0.1):
    '''
        Checks if output from split model is correct 

        return 0 if check fails and 1 if passes
    '''
    
    if comm_layer == 0:
        print(f'Input layer. Skipping comparison')
        return 1
    else:
        # if reaches the end use the full model output 
        if comm_layer == len(horz_output):
            comm_layer_name = 'x'
        else:
            comm_layer_name = layer_names_fx[comm_layer]
    
        truth_output = horz_output[comm_layer_name]
        if torch.is_tensor(truth_output):
            
            max_diff, max_by_Cout = compare_outputs(split_model_output, truth_output)
            if max_diff > 0.1:
                return 0
            else:
                return 1 
        else:
            print(f'Horizontal output is {type(truth_output)}. Skipping comparison')
            return 1


In [17]:
''' Prep Inout for Mock Run version 2 (fx)'''

# TODO: reduce size of communicated tensors to only what is necessary 
# TODO: also check bias for nonzero
# TODO: implement broadcast vs multi-cast? switch e.g. a single machine broadcasts outputs because weights that reach across layers are executed on the rx machine only 
# TODO: handle arbitrary kernel to machine assignment 

# channel_id == INPUTS
# filter_id  == OUTPUTS

configs_copy = configs # keep the origonal config 

# setup input 
N_batch = 1
torch.manual_seed(0)
input_tensor = torch.rand(N_batch, 3, 32, 32, dtype=torch.float64, device=torch.device(configs_copy['device'])) # 1k images, 3 channels, 32x32 image (cifar100) 
model = model.type(torch.float64)

# add some extra logic to handle final splitting of layer 
final_node = 0

# make SplitManagers for split model execution 
debug = True
configs_copy['dtype'] = 'float64'
split_managers = [SplitManager]*num_machines
for i in range(num_machines):
    split_managers[i] = SplitManager(configs_copy, i, num_machines, input_tensor, final_node, debug=True)

# emulates queue that server collects inputs with. 
# Each network node has it's own list of inputs it looks at when collecting inputs 
# input :
#   [rx node (str)] : (list) list of data dicts 
# Write outputs to one big queue with everyone's data in it 
collected_data_queues = {}
for i in range(num_machines):
    in_data = prep_data(input_tensor, -1, 0)
    if split_managers[i].current_layer == 1:
        collected_data_queues[str(i)] =  [in_data]
    else:
        collected_data_queues[str(i)] = []

# put models into eval mode and on device
model.eval()
model.to(configs['device'])

# get true output at each layer
# WARNING: bn layers are initialized differently between model in this notebook and model in split manager. The moving mean and variance fields do not match HOWEVER the weights and biases are the same
#horz_output, layer_output_size_LUT = get_output_at_each_layer(model, input_tensor) 
horz_output, layer_output_size_LUT = get_output_at_each_layer(split_managers[0].model, input_tensor)

'''
    mock run through inference using split models 
'''

# timing
split_execution_start_time = time.time()
layer_completion_time_stamp = {}
layer_execution_duration = {}

# make inference 
with torch.no_grad():

    # iterate through layers 1 module at a time 
    while True:#range(num_total_modules): # 16 <=> layer_1 block 

        # iterate through each machine (done in parallel later)
        for imach in range(num_machines):

            if split_managers[imach].is_done():
                print(f'\tMachine {split_managers[imach].machine} has finished calculations. Skipping...\n')
                continue
            else:
                print(f'\tExecuting on machine {imach}')
            
            # collect communication inputs if necessary 
            # 1. collect inputs for this machine and it's current layer
            # 2. add them to the current_tensor 
            enough_input = split_managers[imach].process_input(collected_data_queues[str(imach)]) # update local tensor with inputs
            if enough_input:
                
                # execute split layers
                output_tensor = split_managers[imach].execute_layers_until_comms()

                # always send output unless on final layer
                if not split_managers[imach].current_layer == split_managers[imach].total_layers_fx:
                    # prep output
                    processed_output = split_managers[imach].prep_output(output_tensor) # prepare communication
                    
                    # Send output! (Append output data to proper rx machine queue)
                    for a_output in processed_output:
                        rx_mach = a_output['node_to']
                        collected_data_queues[str(rx_mach)].append(a_output)

        # are machines finished executing model?
        if all([amanager.is_done() for amanager in split_managers]):
            # final execution time
            tot_split_execution_time = time.time() - split_execution_start_time
            print(f'\n\n############################# FINAL EXECUTION TIME {tot_split_execution_time} [seconds] #############################\n\n')
            break

        # collect all output in one array 
        all_output = [el for alist in list(collected_data_queues.values()) for el in alist]

        # find lowest communication layer that machines are waiting on 
        comm_layer = -1
        for node in range(num_machines):
            node_comm_layer = split_managers[node].current_layer -1
            if comm_layer == -1:
                comm_layer = node_comm_layer
            elif node_comm_layer < comm_layer:
                comm_layer = node_comm_layer
        comm_layer_name = layer_names_fx[comm_layer]
        
        print(f'\n\n---------------------------FINISHED COMMS FOR {comm_layer_name}-----------------------------------\n\n')

        if not comm_layer_name == 'linear':
            # collect communicated output
            split_model_output = SplitManager.combine_all_dict_inputs(
                split_managers[0].get_input_size(comm_layer), 
                all_output, # TODO: assumes you can sum all outputs from the same layer 
                comm_layer, 
                device=split_managers[0].device,
                dtype=split_managers[0].dtype)

            # add current tensors stored in machines 
            for i in range(num_machines):
                if split_managers[i].current_layer-1 == comm_layer:
                    split_model_output = split_model_output + split_managers[i].current_tensor

            # compare split vs full (truth) 
            print(f'\nChecking split model output for {comm_layer_name}:')
            compare_outputs_wrapper(horz_output, comm_layer, split_model_output)
            print('\n\n')



2024-09-15 20:31:41 - INFO - Too few inputs for layer=2
2024-09-15 20:31:41 - DEBUG - 		 Collected inputs from nodes: []
2024-09-15 20:31:41 - DEBUG - 		 Need inputs from nodes: [2. 3.]
2024-09-15 20:31:41 - DEBUG - Current input channels tensor([0, 1, 2])
2024-09-15 20:31:41 - DEBUG - EXECUTED: #1-conv1
2024-09-15 20:31:41 - DEBUG - SPLIT OUTPUT: Shape=[1, 64, 32, 32]; C_out=[16 17 18 19 20 21 22 23 24 25 26 27 28 29 31 56 62]
2024-09-15 20:31:41 - DEBUG - INITIALIZING CURRENT TENSOR FOR: #2-bn1; Shape=[1, 64, 32, 32]; C_in=[16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31]
2024-09-15 20:31:41 - DEBUG - 		Output tensor shape : torch.Size([1, 64, 32, 32])


	Executing on machine 0
	Executing on machine 1


2024-09-15 20:31:41 - DEBUG - Machine=1, Layer to execute = 2:bn1.
2024-09-15 20:31:41 - DEBUG - Prepping to send C_out tensor([56, 62]) to machine 3
2024-09-15 20:31:41 - DEBUG - Current input channels tensor([0, 1, 2])
2024-09-15 20:31:41 - DEBUG - EXECUTED: #1-conv1
2024-09-15 20:31:41 - DEBUG - SPLIT OUTPUT: Shape=[1, 64, 32, 32]; C_out=[ 0 12 38 42 54 57]
2024-09-15 20:31:41 - DEBUG - INITIALIZING CURRENT TENSOR FOR: #2-bn1; Shape=[1, 64, 32, 32]; C_in=[32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
2024-09-15 20:31:41 - DEBUG - 		Output tensor shape : torch.Size([1, 64, 32, 32])
2024-09-15 20:31:41 - DEBUG - Machine=2, Layer to execute = 2:bn1.
2024-09-15 20:31:41 - DEBUG - Prepping to send C_out tensor([ 0, 12]) to machine 0


	Executing on machine 2


2024-09-15 20:31:42 - DEBUG - Machine=2, Layer to execute = 2:bn1.
2024-09-15 20:31:42 - DEBUG - Prepping to send C_out tensor([54, 57]) to machine 3
2024-09-15 20:31:42 - DEBUG - Current input channels tensor([0, 1, 2])
2024-09-15 20:31:42 - DEBUG - EXECUTED: #1-conv1
2024-09-15 20:31:42 - DEBUG - SPLIT OUTPUT: Shape=[1, 64, 32, 32]; C_out=[ 4  8 14 16 17 18 19 20 21 22 23 24 25 26 27 28 29 31 36 42 45 46 47 54
 57]
2024-09-15 20:31:42 - DEBUG - INITIALIZING CURRENT TENSOR FOR: #2-bn1; Shape=[1, 64, 32, 32]; C_in=[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
2024-09-15 20:31:42 - DEBUG - 		Output tensor shape : torch.Size([1, 64, 32, 32])
2024-09-15 20:31:42 - DEBUG - Machine=3, Layer to execute = 2:bn1.
2024-09-15 20:31:42 - DEBUG - Prepping to send C_out tensor([ 4,  8, 14]) to machine 0


...

	Executing on machine 2


2024-09-15 20:31:47 - DEBUG - Machine=2, Layer to execute = 12:layer1.1.bn1.
2024-09-15 20:31:47 - DEBUG - Prepping to send C_out tensor([ 0,  2,  4,  5, 10, 13]) to machine 0
2024-09-15 20:31:47 - DEBUG - Machine=2, Layer to execute = 12:layer1.1.bn1.
2024-09-15 20:31:47 - DEBUG - Prepping to send C_out tensor([19, 20, 21, 22, 24, 30]) to machine 1
2024-09-15 20:31:47 - DEBUG - Machine=2, Layer to execute = 12:layer1.1.bn1.
2024-09-15 20:31:47 - DEBUG - Prepping to send C_out tensor([49, 61, 62]) to machine 3
2024-09-15 20:31:47 - DEBUG - Current input channels tensor([48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63])
2024-09-15 20:31:47 - DEBUG - EXECUTED: #8-layer1.0.bn2
2024-09-15 20:31:47 - DEBUG - SPLIT OUTPUT: Shape=[1, 64, 32, 32]; C_out=[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
2024-09-15 20:31:47 - DEBUG - INITIALIZING CURRENT TENSOR FOR: #9-layer1.0.add; Shape=[1, 64, 32, 32]; C_in=[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
2024-09-15 20:31:47 -

	Executing on machine 3


2024-09-15 20:31:47 - DEBUG - EXECUTED: #11-layer1.1.conv1
2024-09-15 20:31:47 - DEBUG - SPLIT OUTPUT: Shape=[1, 64, 32, 32]; C_out=[ 1  2  3  4  5  6 10 11 22 23 24 30 35 39 41 42 54 56 61]
2024-09-15 20:31:47 - DEBUG - INITIALIZING CURRENT TENSOR FOR: #12-layer1.1.bn1; Shape=[1, 64, 32, 32]; C_in=[48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]
2024-09-15 20:31:47 - DEBUG - 		Output tensor shape : torch.Size([1, 64, 32, 32])
2024-09-15 20:31:47 - DEBUG - Machine=3, Layer to execute = 12:layer1.1.bn1.
2024-09-15 20:31:47 - DEBUG - Prepping to send C_out tensor([ 1,  2,  3,  4,  5,  6, 10, 11]) to machine 0
2024-09-15 20:31:47 - DEBUG - Machine=3, Layer to execute = 12:layer1.1.bn1.
2024-09-15 20:31:47 - DEBUG - Prepping to send C_out tensor([22, 23, 24, 30]) to machine 1


KeyboardInterrupt: 

In [ ]:
layer = 5
machine = 1

print(f'Communication for {split_managers[machine].get_layer_name(layer)} layer:')
for i in range(num_machines):
    tx_nodes = split_managers[i].expected_comms_for_layer(i, layer)
    print(f'\tNode={i}, rx:{tx_nodes}')

print()

Communication for layer1.0.bn1 layer:
	Node=0, rx:[1. 2. 3.]
	Node=1, rx:[0.]
	Node=2, rx:[0. 1. 3.]
	Node=3, rx:[0. 2.]



In [10]:
for i in range(num_machines):
    ending_layer = split_managers[i].get_machine_ending_layer()
    field_ending = split_managers[i].final_layer
    print(f'fucntion {ending_layer}, field {ending_layer}')

fucntion 70, field 70
fucntion 69, field 69
fucntion 69, field 69
fucntion 69, field 69


In [14]:
split_managers[0].final_layer



70